In [1]:
import numpy as np
import math

from quantization import Tensor
from quantization_utils import *
from data_utils import *

# Section: Quantized Mul

---
## To Begin

Let's revisit our quantization schema:

$$lhs_{float} = (lhs_{quant} - lhs_{zp}) * lhs_s$$
$$rhs_{float} = (rhs_{quant} - rhs_{zp}) * rhs_s$$
$$output_{float} = (output_{quant} - output_{zp}) * output_s$$

---
Bring those to our mul implementation:

$$output_{float} = lhs_{float} * rhs_{float}$$
    =>
$$(output_{quant} - output_{zp}) * output_s = (lhs_{quant} - lhs_{zp}) * lhs_s * (rhs_{quant} - rhs_{zp}) * rhs_s$$

=>
$$output_{quant} - output_{zp} = (lhs_{quant} - lhs_{zp}) * (rhs_{quant} - rhs_{zp}) * (\frac{lhs_s * rhs_s}{output_s})$$


Let $$real_s = \frac{lhs_s * rhs_s}{output_s}$$ since we can easily fold this one.

---
Since our goal is to compute output_q, we will have the following equation:


$$output_{quant} = (lhs_{quant} - lhs_{zp}) * (rhs_{quant} - rhs_{zp}) * real_s + output_{zp}$$


So far so good, but just a second, what's the promise of **integer-only**?
Isn't $$real_s$$ a float number? 

---

## Integer-only 

Let's slightly change the representation:

$$real_s = real_{can} * 2^{shift}$$.

$$real_{can}$$ is a canonical representation of the float value, ranges between [0.5, 1), please note our scale is always a positive value, while *shift* is essentially the shift (a left shift if positive, a right shift if negative).

In fact, ```std::frexp``` will help us with the job, see the documentation [here](https://en.cppreference.com/w/cpp/numeric/math/frexp).

Let's just apply one more trick:

$$real_s = \frac{real_{can} * 2^{31} * 2^{shift}}{2^{31}} $$


and $$multiplier = real_{can} * 2^{31}$$

is within the range of int32 representation, rember the canonicacal representation is within [0.5, 1)? so that value is within [2^30, 2^31 - 1], and we
can easily cast that value to int32 with very minimum accuracy lost.

---
## Put it together
Now we know:

$$output_{quant} = \frac{temp* multiplier}{2^{31}} * 2^{shift} + output_{zp}$$

where $$temp = (lhs_{quant} - lhs_{zp}) * (rhs_{quant} - rhs_{zp})$$

Here's the beautilful part:

$$(*) = \frac{temp* multiplier}{2^{31}}$$ is actually pretty *cheap* for the hardware (ARM):

ARM actually has a very verbose name for that: "Signed saturating Rounding Doubling Multiply returning High half." The hardware instruction is acutally just ["VQRDMULH"](http://infocenter.arm.com/help/index.jsp?topic=/com.arm.doc.dui0473j/dom1361289977828.html).

What it does is multiplying the two values then returning the signifiant half (also, rounding).



### Special Note
In previous section, we talked about using int32 for multiplier & temp data, it's also possible to use int16 (but we need to be careful about overflow issue).

In [2]:
lhs_float, lhs_q_tensor = populate_data((4, 4, 4), -4.0, 5.0)
rhs_float, rhs_q_tensor = populate_data((4, 4, 4), -1.5, 0.9)

# ouput ranges (-7.5, 6.0).
# Let's just assume the output has the following:
out_scale = (6.0 + 7.5) / 256.0
out_zp = -(6.0 - 7.5) / (2.0 * out_scale)


In [3]:
def multiply(tensor1, tensor2, output_tensor, datatype=np.int32):
  real_scale = tensor1.scale * tensor2.scale / output_tensor.scale
  multipler_float, shift = np.frexp(real_scale)
  if datatype == np.int32:
    multiplier_shift = 31
  elif datatype == np.int16:
    multiplier_shift = 15
  else:
    assert False
  multiplier = datatype(multipler_float * (2 ** multiplier_shift))

  for i in range(tensor1.value.size):
    lhs_value = datatype(tensor1.value.ravel()[i])
    lhs_value -= tensor1.zero_point
    rhs_value = datatype(tensor2.value.ravel()[i])
    rhs_value -= tensor2.zero_point
    # We need to be careful about the overflow issue when the datatype is int16
    temp = datatype(lhs_value * rhs_value)
    product = saturate_rounding_doubling_multiply(temp, multiplier, datatype)
    output = apply_shift(product, shift)
    output += output_tensor.zero_point
    output = np.int8(min(max(output, -128), 127))
    output_tensor.value.ravel()[i] = output


In [4]:
# First, let's make sure both the inputs are close to our range.
np.allclose(lhs_float, dequant(lhs_q_tensor), atol=5e-2)
np.allclose(rhs_float, dequant(rhs_q_tensor), atol=5e-2)

# Let's check the average deviation as well: avg(abs(a - b))
print("Avg deviation between lhs float and quantized representation is %.3f" % diff(lhs_float, dequant(lhs_q_tensor)))
print("Avg deviation between rhs float and quantized representation is %.3f" % diff(rhs_float, dequant(rhs_q_tensor)))

Avg deviation between lhs float and quantized representation is 0.017
Avg deviation between rhs float and quantized representation is 0.005


In [5]:
# True result
expect_result = lhs_float * rhs_float


# Let's see how the quantized kernel works out:
out_tensor = Tensor(np.zeros(lhs_float.shape), out_scale, out_zp)
multiply(lhs_q_tensor, rhs_q_tensor, out_tensor, np.int32)

# Let's make sure it's within 0.2 range
result = np.allclose(expect_result, dequant(out_tensor), atol=2e-1)
print("With in 0.2 range ", result)

# Also, let's see the average deviation:
print("Avg deviation between output float and quantized representation is %.3f" % diff(expect_result, dequant(out_tensor)))



With in 0.2 range  True
Avg deviation between output float and quantized representation is 0.028


## What's Next?
*   Conv? MatMul?
*   Fixed point?
*   Activations?